<a href="https://colab.research.google.com/github/jcarvaj18643/universidad-Taller1-Recomendacion/blob/main/Recomendacion_Taller1_JaimeCarvajal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlxtend

In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('https://nuelgaloproductos.s3.amazonaws.com/steam-200k.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 5 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   151603712                   199999 non-null  int64  
 1   The Elder Scrolls V Skyrim  199999 non-null  object 
 2   purchase                    199999 non-null  object 
 3   1.0                         199999 non-null  float64
 4   0                           199999 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 7.6+ MB


In [7]:
df.head(10)

,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0
5,151603712,Fallout New Vegas,purchase,1.0,0
6,151603712,Fallout New Vegas,play,12.1,0
7,151603712,Left 4 Dead 2,purchase,1.0,0
8,151603712,Left 4 Dead 2,play,8.9,0
9,151603712,HuniePop,purchase,1.0,0


In [5]:
column_names = ['User ID', 'Game Name', 'Behavior Name', 'Value', 'Unknown']

df.columns = column_names

# Eliminamos la columna 'Unknown' ya que no parece relevante
df = df.drop(columns=['Unknown'])

# Verificamos los primeros registros para asegurarnos de que las columnas están correctamente asignadas
df.head()

,User ID,Game Name,Behavior Name,Value
0,151603712,The Elder Scrolls V Skyrim,play,273.0
1,151603712,Fallout 4,purchase,1.0
2,151603712,Fallout 4,play,87.0
3,151603712,Spore,purchase,1.0
4,151603712,Spore,play,14.9


In [6]:
df_purchases = df[df['Behavior Name'] == 'purchase']

# Pivotamos la tabla para tener una matriz de usuarios por juegos
basket = df_purchases.pivot_table(index='User ID', columns='Game Name', values='Value', aggfunc='sum').fillna(0)

# Convertimos en valores binarios (1 si el usuario compró el juego, 0 si no lo hizo)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# Mostramos la matriz resultante
basket.head()

Game Name,007 Legends,0RBITALIS,1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby),10 Second Ninja,"10,000,000",100% Orange Juice,1000 Amps,12 Labours of Hercules,12 Labours of Hercules II The Cretan Bull,12 Labours of Hercules III Girl Power,...,rFactor 2,realMyst,realMyst Masterpiece Edition,resident evil 4 / biohazard 4,rymdkapsel,sZone-Online,samurai_jazz,the static speaks my name,theHunter,theHunter Primal
User ID,,,,,,,,,,,,,,,,,,,,,
5250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
76767,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86540,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144736,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# Mostramos los primeros conjuntos de juegos frecuentes
frequent_itemsets.head()

,support,itemsets
0,0.021141,(APB Reloaded)
1,0.013153,(ARK Survival Evolved)
2,0.017913,(AdVenture Capitalist)
3,0.019204,(Age of Empires II HD Edition)
4,0.010490,(Alan Wake)


In [12]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5).sort_values(by = ['confidence', 'zhangs_metric'],
                                                                             ascending=[False, False])

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
18,(Arma 2 Operation Arrowhead Beta (Obsolete)),(Arma 2 Operation Arrowhead),0.020173,0.020173,0.020173,1.0,49.572000,0.019766,inf,1.000000
19,(Arma 2 Operation Arrowhead),(Arma 2 Operation Arrowhead Beta (Obsolete)),0.020173,0.020173,0.020173,1.0,49.572000,0.019766,inf,1.000000
28,(Arma 3 Zeus),(Arma 3),0.012991,0.012991,0.012991,1.0,76.975155,0.012822,inf,1.000000
29,(Arma 3),(Arma 3 Zeus),0.012991,0.012991,0.012991,1.0,76.975155,0.012822,inf,1.000000
86,(Call of Duty Black Ops),(Call of Duty Black Ops - Multiplayer),0.020495,0.020495,0.020495,1.0,48.791339,0.020075,inf,1.000000
...,...,...,...,...,...,...,...,...,...,...
88017,"(Half-Life, Half-Life Opposing Force, Half-Lif...","(Half-Life 2 Lost Coast, Counter-Strike Source)",0.028726,0.052449,0.014363,0.5,9.533077,0.012856,1.895102,0.921575
400,(Metro 2033),(The Elder Scrolls V Skyrim),0.025176,0.057775,0.012588,0.5,8.654330,0.011133,1.884451,0.907292
3666,"(Terraria, Team Fortress 2)",(The Elder Scrolls V Skyrim),0.024853,0.057775,0.012426,0.5,8.654330,0.010991,1.884451,0.906992
222,(Don't Starve Together Beta),(Garry's Mod),0.020657,0.058985,0.010328,0.5,8.476744,0.009110,1.882030,0.900634


**FP-Growth (Frequent Pattern Growth)**

In [13]:
from mlxtend.frequent_patterns import fpgrowth

# Aplicamos el algoritmo FP-Growth
frequent_itemsets_fp = fpgrowth(basket, min_support=0.01, use_colnames=True)

# Generamos las reglas de asociación usando FP-Growth
rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.5)

# Ordenamos las reglas por confianza
rules_fp.sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
540104,"(Borderlands DLC The Zombie Island of Dr. Ned,...",(Borderlands DLC The Secret Armory of General ...,0.010167,0.011135,0.010167,1.0,89.804348,0.010054,inf,0.999022
540097,"(Borderlands DLC The Zombie Island of Dr. Ned,...",(Borderlands DLC The Secret Armory of General ...,0.010167,0.011135,0.010167,1.0,89.804348,0.010054,inf,0.999022
389998,"(Day of Defeat, Deathmatch Classic, Half-Life ...","(Half-Life, Team Fortress Classic, Ricochet, H...",0.012346,0.013717,0.012346,1.0,72.900000,0.012176,inf,0.998611
540284,"(Arma 2, Arma 2 Operation Arrowhead, Arma 2 Da...",(Arma 2 Operation Arrowhead Beta (Obsolete)),0.010167,0.020173,0.010167,1.0,49.572000,0.009962,inf,0.989892
145477,"(Half-Life, Half-Life 2, Deathmatch Classic, R...","(Half-Life Opposing Force, Counter-Strike)",0.011539,0.024611,0.011539,1.0,40.632787,0.011255,inf,0.986776
...,...,...,...,...,...,...,...,...,...,...
226741,"(Counter-Strike, Half-Life Blue Shift)","(Half-Life 2 Lost Coast, Half-Life Opposing Fo...",0.024691,0.012346,0.012346,0.5,40.500000,0.012041,1.975309,1.000000
526471,(Arma 2 Operation Arrowhead Beta (Obsolete)),"(Garry's Mod, Arma 2, Arma 2 Operation Arrowhead)",0.020173,0.010086,0.010086,0.5,49.572000,0.009883,1.979827,1.000000
526472,(Arma 2 Operation Arrowhead),"(Arma 2 Operation Arrowhead Beta (Obsolete), A...",0.020173,0.010086,0.010086,0.5,49.572000,0.009883,1.979827,1.000000
40924,"(Half-Life, Counter-Strike, Ricochet, Deathmat...","(Half-Life 2, Half-Life 2 Lost Coast, Day of D...",0.023723,0.011942,0.011862,0.5,41.868243,0.011578,1.976116,0.999835


**Recomendacion por FP-Growth **

In [14]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

frequent_itemsets_fp = fpgrowth(basket, min_support=0.01, use_colnames=True)

print(frequent_itemsets_fp.head())

rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.5)

rules_fp_sorted = rules_fp.sort_values(by=['confidence', 'lift'], ascending=[False, False])

print(rules_fp_sorted.head())

    support                  itemsets
0  0.390624                  (Dota 2)
1  0.187445         (Team Fortress 2)
2  0.079158  (Half-Life 2 Lost Coast)
3  0.078916   (Counter-Strike Source)
4  0.069071          (Counter-Strike)
                                              antecedents  \
167369  (Half-Life, Counter-Strike Condition Zero Dele...   
167371  (Half-Life, Team Fortress Classic, Counter-Str...   
167403  (Half-Life, Counter-Strike Condition Zero Dele...   
167408  (Half-Life, Counter-Strike, Team Fortress Clas...   
167418  (Half-Life, Counter-Strike Condition Zero Dele...   

                                              consequents  antecedent support  \
167369  (Half-Life Opposing Force, Counter-Strike Cond...            0.010248   
167371  (Half-Life Opposing Force, Counter-Strike Cond...            0.010248   
167403  (Half-Life Opposing Force, Counter-Strike Cond...            0.010248   
167408  (Half-Life Opposing Force, Counter-Strike Cond...            0.010248   


In [15]:
import pickle

# Guardar el modelo Apriori
with open('apriori_rules.pkl', 'wb') as f:
    pickle.dump(rules_fp, f)

# Guardar el modelo FP-Growth
with open('fpgrowth_rules.pkl', 'wb') as f:
    pickle.dump(rules_fp_sorted, f)

In [16]:
from google.colab import files

# Descargar el archivo Apriori
files.download('apriori_rules.pkl')

# Descargar el archivo FP-Growth
files.download('fpgrowth_rules.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>